# Data Enrichment
Data enrichment is the process of filling in lost data or adding data from both internal and external sources by correlating based on certain columns so that data analysis is sharper.

For example, an empty Year Value Shopping data can be filled with the mean value of the whole data.

Another example, blank postcode data can be filled in if we have a postal code master. Or by searching from another address that is similar and filled in with the zip code.

In [24]:
library(openxlsx)
data.pelanggan <- read.xlsx("C:\\Users\\aftermath\\Documents\\CS-101\\R\\asset\\belanja.pelanggan.xlsx")
data.pelanggan

kode_pelanggan,nilai_belanja_setahun
KD-00032,1275600.0
KD-00053,317800.0
KD-00133,1537200.0
KD-00056,1524700.0
KD-00111,655400.0
KD-00036,1444400.0
KD-00126,350400.0
KD-00137,354600.0
KD-00046,541300.0
KD-00027,536000.0


### Replace the Missing Value with Mean

In [25]:
data.pelanggan$nilai_belanja_setahun <- as.numeric(data.pelanggan$nilai_belanja_setahun)
summary(data.pelanggan)

 kode_pelanggan     nilai_belanja_setahun
 Length:155         Min.   : 237400      
 Class :character   1st Qu.: 504800      
 Mode  :character   Median : 851600      
                    Mean   : 857227      
                    3rd Qu.:1179800      
                    Max.   :1537200      
                    NA's   :4            

In [26]:
mean.belanja <- mean(data.pelanggan$nilai_belanja_setahun, na.rm = TRUE)

data.pelanggan$nilai_belanja_setahun[is.na(data.pelanggan$nilai_belanja_setahun)] <- mean.belanja

In [27]:
summary(data.pelanggan)

 kode_pelanggan     nilai_belanja_setahun
 Length:155         Min.   : 237400      
 Class :character   1st Qu.: 520000      
 Mode  :character   Median : 857227      
                    Mean   : 857227      
                    3rd Qu.:1168250      
                    Max.   :1537200      

In [28]:
write.xlsx(data.pelanggan, file="C:\\Users\\aftermath\\Documents\\CS-101\\R\\asset\\staging.enrichment.nilai_belanja.xlsx")

### Replace the Missing Value with Median

In [10]:
data.pelanggan$nilai_belanja_setahun <- as.numeric(data.pelanggan$nilai_belanja_setahun)
summary(data.pelanggan)

 kode_pelanggan     nilai_belanja_setahun
 Length:155         Min.   : 237400      
 Class :character   1st Qu.: 504800      
 Mode  :character   Median : 851600      
                    Mean   : 857227      
                    3rd Qu.:1179800      
                    Max.   :1537200      
                    NA's   :4            

In [12]:
median.belanja <- median(data.pelanggan$nilai_belanja_setahun, na.rm = TRUE)


[1] 851600

In [14]:
data.pelanggan$nilai_belanja_setahun[is.na(data.pelanggan$nilai_belanja_setahun)] <- median.belanja
summary(data.pelanggan)

 kode_pelanggan     nilai_belanja_setahun
 Length:155         Min.   : 237400      
 Class :character   1st Qu.: 520000      
 Mode  :character   Median : 851600      
                    Mean   : 857081      
                    3rd Qu.:1168250      
                    Max.   :1537200      

### Replace with Similarity
The first thing we do is look for duplicates. For example in this practice we use just one column, which is the address and with a tighter threshold, the number 10.

The second thing we need to do is look for the grouping number of the postal code that is empty. Then based on grouping, we take a non-empty zip code.

In [15]:
library(stringdist)

In [20]:
data.pelanggan <- read.xlsx("C:\\Users\\aftermath\\Documents\\CS-101\\R\\asset\\merged.data2.xlsx")
str(data.pelanggan)

'data.frame':	155 obs. of  9 variables:
 $ kode_pelanggan    : chr  "KD-00001" "KD-00002" "KD-00003" "KD-00004" ...
 $ nama              : chr  "Agus Cahyonos" "Khairul Nissa" "Slamet Wiyanto" "DRS. Maria Simangunsong" ...
 $ alamat            : chr  "Jalan Pulo Bambu No. 15, Kota Tenggara Lama" "Taman Vivo Indah, Blok AA No. 7" "Meta Residences, No. 32C" "Gang Bulan Desember III, No. 9" ...
 $ no_telepon        : chr  "08298911112222" "+6287132221371404" "+6285725955303368" "+6283376770990635" ...
 $ anomali_no_telepon: logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ kode_pos          : chr  "876511" "712983" "764550" "967220" ...
 $ tanggal_lahir     : chr  "08-02-1967" "23-10-1991" "23-11-1962" "17-02-2097" ...
 $ umur              : num  51.2 26.5 55.5 -78.9 31.7 ...
 $ umur_valid        : logi  TRUE TRUE TRUE TRUE TRUE TRUE ...


In [21]:
#------------------- 1. PERSIAPAN: MENEMUKAN DUPLIKASI ALAMAT -----------------------
#Konversi Factor
data.pelanggan$kode_pos <- as.factor(data.pelanggan$kode_pos)

#Inisialiasi variable grouping_no dengan nilai 1
grouping_no <- 1

#Inisialisasi hasil.akhir
hasil.akhir <- NULL

while(length(data.pelanggan$alamat)>0)
{
  #Variable referensi alamat diambil dari item pertama
  referensi.alamat <- data.pelanggan$alamat[1]
  
  #Menghitung jarak antara referensi dengan item alamat dengan method "lv" 
  jarak.teks.alamat <- stringdist(referensi.alamat, data.pelanggan$alamat, method="lv")

  #Hasil filter jarak dengan threshold 
  # - lebih kecil dari angka 10 untuk alamat
  #disimpan ke variable filter.jarak
  filter.jarak <- (jarak.teks.alamat < 10)

  #Melakukan filtering pada variable data.pelanggan, dan mengambil dua kolom 
  #untuk disimpan ke dua variable 
  kode_pelanggan.temp <- data.pelanggan[filter.jarak,]$kode_pelanggan
  alamat.temp <- data.pelanggan[filter.jarak,]$alamat
  kode_pos.temp <- data.pelanggan[filter.jarak,]$kode_pos
  
  #Konstruksi temporary variable
  var.temp <- data.frame(grouping=grouping_no, kode_pelanggan=kode_pelanggan.temp, kode_pos=kode_pos.temp, alamat=alamat.temp, jumlah_record=length(kode_pelanggan.temp))

  #Menggabungkan temporary variable dengan hasil sebelumnya
  hasil.akhir <- rbind(hasil.akhir, var.temp)
  
  #Membuang porsi yang sudah ditemukan
  data.pelanggan <- data.pelanggan[!filter.jarak,]

  #Menambahkan nilai grouping untuk diambil pada iterasi selanjutnya
  grouping_no <- grouping_no + 1
}
hasil.akhir
summary(hasil.akhir$kode_pos)

grouping,kode_pelanggan,kode_pos,alamat,jumlah_record
1,KD-00001,876511,"Jalan Pulo Bambu No. 15, Kota Tenggara Lama",4
1,KD-00012,876511,"Pulo Bambu No. 15, Kota Tenggara Lama",4
1,KD-00045,876511,"Pulo Bambu No. 57, Kota Tenggara Lama",4
1,KD-00778,876511,Jalan Pulau Bambu No. 15 - Kota Tenggara Lama,4
2,KD-00002,712983,"Taman Vivo Indah, Blok AA No. 7",2
2,KD-00075,712983,"Taman Vivo Indah, Blok AA No. 7",2
3,KD-00003,764550,"Meta Residences, No. 32C",2
3,KD-00043,764550,"Meta Residences, No. 1A",2
4,KD-00004,967220,"Gang Bulan Desember III, No. 9",3
4,KD-00071,967220,"Gang Bulan Desember III, No. 155",3


- 321321 349922 349981 476511 476533 487451 487851 511431 511432 567120 
     5      8      1      2      2      2      1      1      2      2      3 
567130 567151 633429 633430 633431 666122 666123 696193 712983 712984 764449 
     3      2      2      1      1      3      1      9      2      6      2 
764450 764550 768031 768034 768035 768091 811613 813442 813444 817321 817324 
     2      2      2      2      1      2      3      3      1      1      1 
866162 876511 876512 876551 876552 876612 876614 877521 877613 877614 877615 
     1      6      1      2      2      2      1      3      1      1      1 
893422 896112 896113 896114 896115 896549 896550 896555 896566 967220 967221 
     1      1      1      1      1      8      2      7      1      3      2 
967222 967223 967224 967229 986454 986455 986456 987451 987452 987453 987601 
     1      3      1      2      1      8      2      1      2      2      2

In [22]:
#----------------- 2. MENGISI KODE POS YANG KOSONG --------------------
#Inisialisasi kolom baru kode pos
hasil.akhir$kode_pos_enrich <-hasil.akhir$kode_pos 

#Mengambil nomor grouping kode pos yang kosong
kode_pos_kosong <- hasil.akhir[hasil.akhir$kode_pos == "-",]

while(length(kode_pos_kosong$kode_pos)>0)
{
	grouping_no <- kode_pos_kosong$grouping[1]
	#Membuat variable filter 
	filter.data <- hasil.akhir$grouping == grouping_no & hasil.akhir$kode_pos != "-" 
	#Mengambil data pertama dari hasil filter dengan function head
	temp.data <- head(hasil.akhir[filter.data,],1)
  	#Mengisi kolom kode_pos_enrich dengan kolom kode_pos yang ditemukan
hasil.akhir[hasil.akhir$grouping == grouping_no,]$kode_pos_enrich <- temp.data$kode_pos
  
    #Menghapus row pertama dari variable kode_pos_kosong
    kode_pos_kosong <- kode_pos_kosong[-1,]
}
hasil.akhir
summary(hasil.akhir$kode_pos_enrich)


grouping,kode_pelanggan,kode_pos,alamat,jumlah_record,kode_pos_enrich
1,KD-00001,876511,"Jalan Pulo Bambu No. 15, Kota Tenggara Lama",4,876511
1,KD-00012,876511,"Pulo Bambu No. 15, Kota Tenggara Lama",4,876511
1,KD-00045,876511,"Pulo Bambu No. 57, Kota Tenggara Lama",4,876511
1,KD-00778,876511,Jalan Pulau Bambu No. 15 - Kota Tenggara Lama,4,876511
2,KD-00002,712983,"Taman Vivo Indah, Blok AA No. 7",2,712983
2,KD-00075,712983,"Taman Vivo Indah, Blok AA No. 7",2,712983
3,KD-00003,764550,"Meta Residences, No. 32C",2,764550
3,KD-00043,764550,"Meta Residences, No. 1A",2,764550
4,KD-00004,967220,"Gang Bulan Desember III, No. 9",3,967220
4,KD-00071,967220,"Gang Bulan Desember III, No. 155",3,967220


- 321321 349922 349981 476511 476533 487451 487851 511431 511432 567120 
     0      8      1      2      2      2      1      1      2      2      3 
567130 567151 633429 633430 633431 666122 666123 696193 712983 712984 764449 
     3      2      2      1      1      3      1      8      2      6      2 
764450 764550 768031 768034 768035 768091 811613 813442 813444 817321 817324 
     2      2      2      2      1      3      3      3      1      2      1 
866162 876511 876512 876551 876552 876612 876614 877521 877613 877614 877615 
     3      6      1      2      2      2      1      3      1      1      1 
893422 896112 896113 896114 896115 896549 896550 896555 896566 967220 967221 
     1      1      1      1      1      8      2      7      1      3      2 
967222 967223 967224 967229 986454 986455 986456 987451 987452 987453 987601 
     1      3      1      2      1      9      2      1      2      3      2

In [23]:
write.xlsx(hasil.akhir, file="C:\\Users\\aftermath\\Documents\\CS-101\\R\\asset\\staging.enrichment.kode_pos.xlsx")

Note: zip::zip() is deprecated, please use zip::zipr() instead


### Merge Enriched Data

In [30]:
mean.data <- read.xlsx("C:\\Users\\aftermath\\Documents\\CS-101\\R\\asset\\staging.enrichment.nilai_belanja.xlsx")
postal.data <- read.xlsx("C:\\Users\\aftermath\\Documents\\CS-101\\R\\asset\\staging.enrichment.kode_pos.xlsx")

In [31]:
final.enriched <- merge(x = mean.data, y = postal.data, by.x = "kode_pelanggan", by.y = "kode_pelanggan", all = TRUE)
final.enriched

kode_pelanggan,nilai_belanja_setahun,grouping,kode_pos,alamat,jumlah_record,kode_pos_enrich
KD-00001,1082900,1,876511,"Jalan Pulo Bambu No. 15, Kota Tenggara Lama",4,876511
KD-00002,1336200,2,712983,"Taman Vivo Indah, Blok AA No. 7",2,712983
KD-00003,601700,3,764550,"Meta Residences, No. 32C",2,764550
KD-00004,451500,4,967220,"Gang Bulan Desember III, No. 9",3,967220
KD-00005,1488900,5,476511,"Jalan Tegal Sari Indah, No. D87 -- Kota H",2,476511
KD-00006,257100,6,487851,"Perum Pluto, Blok C No. 1",1,487851
KD-00007,805900,7,986455,"Apartemen Kecapi Indah, Lt. 16 No. 1610",2,986455
KD-00008,879800,8,813444,"Kali Mars Cluster, No. 24C",2,813444
KD-00009,272600,9,896555,"Jalan Kebon Jahe, No. F16 - Kota E",1,896555
KD-00010,389400,10,987453,"Perum Venus, Gg. Harimau No. 1A",3,987453


In [32]:
write.xlsx(file = "C:\\Users\\aftermath\\Documents\\CS-101\\R\\asset\\merged.enriched.data.xlsx", final.enriched)